In [27]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [29]:
%autoreload
from pathlib import Path
import sys
root = Path().resolve().absolute().parent.parent
print(root)
sys.path.append(str(root))

from src.pipeline import Pipeline, BuilingIdsEnum
pipe = Pipeline()



/Users/matsalexander/Desktop/SolarEnergyImpact
Index(['timestamp', 'value_import', 'property_id', 'building', 'area',
       'value_export', 'solar_consumption', 'predicted_consumption',
       'temperature', 'wind_speed', 'wind_direction', 'cloud_fraction',
       'precipitation'],
      dtype='object')


In [30]:
building_a = pipe.get_data(BuilingIdsEnum.A)
building_b = pipe.get_data(BuilingIdsEnum.B)
building_c = pipe.get_data(BuilingIdsEnum.C)

# remove from 3. to 7. july 2024 from dataset A
mask = (building_a['timestamp'] >= '2024-07-03') & (building_a['timestamp'] <= '2024-07-07')
building_a = building_a[~mask]

combined_df = pd.concat([building_a, building_b, building_c])
# reset index
combined_df = combined_df.reset_index(drop=True)
# Perform the train-test split with stratification based on 'building_id'
train_data, test_data = train_test_split(
    combined_df,
    test_size=0.20,
    random_state=42,
    stratify=combined_df['building']
)

# select features
features = ["timestamp", "area", "temperature", "wind_speed", "cloud_fraction", "precipitation"]
target = "value_import"

train_data = train_data[features + [target]]
test_data = test_data[features + [target]]



In [33]:
# Train the model using AutoGluon
predictor = TabularPredictor(label=target, eval_metric='mean_absolute_error').fit(
    train_data, 
    presets='best_quality',
    excluded_model_types=['KNN']
    )

No path specified. Models will be saved in: "AutogluonModels/ag-20241109_185606"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       13.88 GB / 32.00 GB (43.4%)
Disk Space Avail:   618.51 GB / 926.35 GB (66.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal 

[1000]	valid_set's l1: 5.92006
[2000]	valid_set's l1: 5.66781
[3000]	valid_set's l1: 5.51892
[4000]	valid_set's l1: 5.41641
[5000]	valid_set's l1: 5.34046
[6000]	valid_set's l1: 5.27797
[7000]	valid_set's l1: 5.22588
[8000]	valid_set's l1: 5.18609
[9000]	valid_set's l1: 5.15965
[10000]	valid_set's l1: 5.13844
[1000]	valid_set's l1: 5.89552
[2000]	valid_set's l1: 5.64629
[3000]	valid_set's l1: 5.50757
[4000]	valid_set's l1: 5.412
[5000]	valid_set's l1: 5.34405
[6000]	valid_set's l1: 5.28655


	Ran out of time, early stopping on iteration 6791. Best iteration is:
	[6790]	valid_set's l1: 5.2483


[1000]	valid_set's l1: 5.89883
[2000]	valid_set's l1: 5.62877
[3000]	valid_set's l1: 5.47533
[4000]	valid_set's l1: 5.36764
[5000]	valid_set's l1: 5.29681


	Ran out of time, early stopping on iteration 5428. Best iteration is:
	[5428]	valid_set's l1: 5.2738


...

	Ran out of time, early stopping on iteration 7645. Best iteration is:
	[7331]	valid_set's l1: 4.22771


[1000]	valid_set's l1: 4.54051
[2000]	valid_set's l1: 4.36481
[3000]	valid_set's l1: 4.28381
[4000]	valid_set's l1: 4.25482
[5000]	valid_set's l1: 4.23095
[6000]	valid_set's l1: 4.22501
[7000]	valid_set's l1: 4.22529
[1000]	valid_set's l1: 4.5333
[2000]	valid_set's l1: 4.32893
[3000]	valid_set's l1: 4.23856
[4000]	valid_set's l1: 4.19375
[5000]	valid_set's l1: 4.16914
[6000]	valid_set's l1: 4.15592
[7000]	valid_set's l1: 4.14293
[8000]	valid_set's l1: 4.13753
[9000]	valid_set's l1: 4.13851


	Ran out of time, early stopping on iteration 9862. Best iteration is:
	[9825]	valid_set's l1: 4.1341


[1000]	valid_set's l1: 4.52472
[2000]	valid_set's l1: 4.34443
[3000]	valid_set's l1: 4.26338
[4000]	valid_set's l1: 4.2182
[5000]	valid_set's l1: 4.19852
[6000]	valid_set's l1: 4.18526
[7000]	valid_set's l1: 4.1845
[8000]	valid_set's l1: 4.18805
[1000]	valid_set's l1: 4.43207
[2000]	valid_set's l1: 4.2146
[3000]	valid_set's l1: 4.12805
[4000]	valid_set's l1: 4.07309
[5000]	valid_set's l1: 4.03687
[6000]	valid_set's l1: 4.01975
[7000]	valid_set's l1: 4.00471
[8000]	valid_set's l1: 3.99838
[9000]	valid_set's l1: 4.00485
[10000]	valid_set's l1: 4.01093
[1000]	valid_set's l1: 4.45086
[2000]	valid_set's l1: 4.26881
[3000]	valid_set's l1: 4.19756
[4000]	valid_set's l1: 4.16446
[5000]	valid_set's l1: 4.14628
[6000]	valid_set's l1: 4.14811


	-4.1577	 = Validation score   (-mean_absolute_error)
	753.37s	 = Training   runtime
	2.3s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 155.0s of the 1054.47s of remaining time.
	-3.4913	 = Validation score   (-mean_absolute_error)
	3.72s	 = Training   runtime
	0.69s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 150.19s of the 1049.67s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 3826.
	Ran out of time, early stopping on iteration 3947.
	Ran out of time, early stopping on iteration 3957.
	Ran out of time, early stopping on iteration 4136.
	Ran out of time, early stopping on iteration 4257.
	Ran out of time, early stopping on iteration 4417.
	Ran out of time, early stopping on iteration 4630.
	Ran out of time, early stopping on iteration 5149.
	-4.4949	 = Validation score   (-mean_absolute_error)
	144.

In [ ]:
# AutogluonModels/ag-20241109_185606
# predictor = TabularPredictor.load("AutogluonModels/ag-20241109_185606")
# Evaluate on test data
performance = predictor.evaluate(test_data)
# best model: ag-20241022_161331

print("Evaluation Performance:")
performance
# reset index 
test_data = test_data.reset_index(drop=True)
# To see feature importance
global_importance = predictor.feature_importance(test_data)
print("\nFeature Importance:")
print(global_importance)  # Shows which features had the most impact on model predictions


Computing feature importance via permutation shuffling for 6 features using 5000 rows with 5 shuffle sets...


Evaluation Performance:


	470.48s	= Expected runtime (94.1s per shuffle set)
	379.88s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                importance    stddev       p_value  n  p99_high   p99_low
area              5.598706  0.073723  3.607004e-09  5  5.750503  5.446909
timestamp         5.322324  0.073009  4.247822e-09  5  5.472650  5.171998
temperature       4.483050  0.063063  4.697688e-09  5  4.612898  4.353201
wind_speed        0.673139  0.029942  4.685364e-07  5  0.734791  0.611488
cloud_fraction    0.503317  0.029003  1.317308e-06  5  0.563036  0.443599
precipitation     0.172484  0.010950  1.939006e-06  5  0.195031  0.149937


In [35]:
# model location => AutogluonModels/ag-20241016_095906
main_building = pipe.get_data(BuilingIdsEnum.MAIN)

data_predict = main_building[features]
data_predict


,timestamp,area,temperature,wind_speed,cloud_fraction,precipitation
0,2023-07-01 00:00:00,1199,13.6,1.6,0.5,0.0
1,2023-07-01 01:00:00,1199,13.2,2.0,0.4,0.0
2,2023-07-01 02:00:00,1199,12.3,1.6,0.3,0.0
3,2023-07-01 03:00:00,1199,11.9,0.6,0.9,0.0
4,2023-07-01 04:00:00,1199,11.9,0.2,0.8,0.0
...,...,...,...,...,...,...
10338,2024-09-03 18:00:00,1199,17.8,2.5,1.0,1.4
10339,2024-09-03 19:00:00,1199,17.8,1.8,1.0,2.5
10340,2024-09-03 20:00:00,1199,17.7,1.1,1.0,2.6
10341,2024-09-03 21:00:00,1199,18.0,3.4,1.0,2.9


In [36]:
prediciton1 = predictor.predict(data_predict)

In [37]:
# save predicitons as a csv in data folder from root.
import datetime
from pathlib import Path


prediciton1_df = pd.DataFrame(prediciton1)
date_time = datetime.datetime.now().strftime("%Y%m%d")
my_path = Path().resolve().parent.parent / 'data'/ "pred" / 'prediction_drita2.csv'
# create folder
my_path.parent.mkdir(parents=True, exist_ok=True)
prediciton1_df.to_csv(my_path, index=False)